In [1]:
# Import selenium dependencies
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException


/Users/jdl/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Import other dependencies
import time
import requests
from bs4 import BeautifulSoup
import re
import config
import json
import os
from urllib.parse import urlparse
import csv
from dotenv import load_dotenv
import random

In [3]:
video_counter = 254

In [19]:
# setup chromedriver
driver = webdriver.Chrome()

# open the webpage
driver.get("https://www.instagram.com")

In [20]:
load_dotenv()
username_value = os.getenv("USERNAME")
password_value = os.getenv("PASSWORD")
# target username
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))

# enter username and password
username.clear()
username.send_keys(username_value)
password.clear()
password.send_keys(password_value)

# target the login button and click it
button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

In [42]:
# Ensure the website URL is set to Instagram's homepage
if driver.current_url != f"https://www.instagram.com/{username_value}/":
    driver.get(f"https://www.instagram.com/{username_value}/")

# Wait up to 10 seconds for the search button to be clickable
search_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, 'svg[aria-label="Search"]'))
)

# Click the search button once it becomes clickable
time.sleep(0.5)
search_button.click()
time.sleep(0.5)

In [43]:
# target the search input field
searchbox = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='Search']")))
searchbox.clear()
# search for the @handle or keyword
keyword = '@apple' #define the account we're looking for
searchbox.send_keys(keyword)
time.sleep(1)

In [44]:
# Check if the keyword starts with "@"
if keyword.startswith("@"):
    keyword = keyword[1:]  # Remove @ symbol

# Wait for the search results to load
first_result = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, f'//span[text()="{keyword}"]'))
)

# Wait until the result becomes clickable
first_result = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, f'//span[text()="{keyword}"]'))
)

# Click the found element
first_result.click()
time.sleep(1.5)

In [45]:
# Get the initial page height
initial_height = driver.execute_script("return document.body.scrollHeight")
reel_urls = set()

while True:
    # Parse URLs of only reels
    anchors = driver.find_elements(By.TAG_NAME, "a")
    
    for anchor in anchors:
        href = anchor.get_attribute("href")
        if href and "/reel/" in href:
            reel_urls.add(href)
        
        # Stop collecting once we have 10 reels
        if len(reel_urls) >= 10:
            break  

    # Stop scrolling if we have reached 10 reels
    if len(reel_urls) >= 10:
        break  

    # Scroll down and check if more content loads
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    WebDriverWait(driver, 10).until(
        lambda d: d.execute_script("return document.body.scrollHeight") > initial_height
    )

    # Update height after new content loads
    initial_height = driver.execute_script("return document.body.scrollHeight")

print(f"Collected {len(reel_urls)} unique reels.")

Collected 10 unique reels.


In [46]:
print(len(reel_urls))

10


In [47]:
# Create a directory called "videos" to save reels
video_dir = "ads"
os.makedirs(video_dir, exist_ok=True)

# Download videos
query_parameters = "__a=1&__d=dis"
json_list = []

for url in reel_urls:
    try:
        modified_url = url + "?" + query_parameters
        driver.get(modified_url)
        time.sleep(random.uniform(2, 5))

        # Extract JSON data
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//pre'))
        )
        json_script = driver.find_element(By.XPATH, '//pre').text
        json_parsed = json.loads(json_script)

        # Extract video URL
        item = json_parsed.get('items', [])[0]
        video_versions = item.get('video_versions', [])
        if video_versions:
            video_url = video_versions[0].get('url')
            json_list.append(video_url)

    except (NoSuchElementException, TimeoutException, json.JSONDecodeError) as e:
        print(f"Error processing reel {url}: {e}")

In [48]:
# Download each reel video
for url in json_list:
    time.sleep(random.uniform(2, 5))
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        file_name = f"ad-{video_counter}.mp4"
        file_path = os.path.join(video_dir, file_name)
        with open(file_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print(f"Downloaded: {file_path}")
        video_counter += 1
    else:
        print(f"Failed to download: {url}")

print(f"Downloaded {len(json_list)} reels to {video_dir}")

Downloaded: ads/ad-294.mp4
Downloaded: ads/ad-295.mp4
Downloaded: ads/ad-296.mp4
Downloaded: ads/ad-297.mp4
Downloaded: ads/ad-298.mp4
Downloaded: ads/ad-299.mp4
Downloaded: ads/ad-300.mp4
Downloaded: ads/ad-301.mp4
Downloaded: ads/ad-302.mp4
Downloaded: ads/ad-303.mp4
Downloaded 10 reels to ads
